#### The Global State of Social Connections

Fielding for this global survey study took place as part of Gallup’s annual World Poll survey administration. Surveys were administered from June 2022 through February 2023, with approximately 1,000 participants over the age of 15 interviewed in 142 countries. In Iceland, Jamaica and Puerto Rico, 500 individuals were interviewed. In Algeria, the number of respondents was 821, and in India and Russia, the number of respondents was 2,006 and 3,000, respectively. In all other countries and areas, the number of respondents ranged from 1,000 to 1,046. Data collection happened over the phone in countries where phone penetration is high (approximately >80% of the population) and face-to-face otherwise. 

Available Datasets:

https://www.gallup.com/analytics/509675/state-of-social-connections.aspx#jumpAlt1

Asked questions:

In general, how connected do you feel to people? By connected, I mean how close you feel to people
emotionally. Do you feel…?2
* Very connected
* Fairly connected
* A little connected
* Not at all connected
* (Don't know)/(Refused)

In general, how lonely do you feel? By lonely, I mean how much you feel emotionally isolated from
people. Do you feel…?
* Very lonely
* Fairly lonely
* A little lonely
* Not at all lonely
* (Don't know)/(Refused)

In [1]:
# Import required modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sql_functions
import numpy as np 
import pandas as pd 
import plotly
import plotly.express as px
from sql_functions import *
from some_functions import *

python(52134) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [2]:
schema = 'cgn_analytics_24_1'
comb_df = get_dataframe(f"Select * from {schema}.mindscape_df_master")
comb_df_copy = comb_df.copy()
unique_country = comb_df_copy.drop_duplicates(subset=['country', 'code'])
code_dict = dict(zip(unique_country['country'], unique_country['code']))
code_list = list(code_dict.values())

In [3]:
schema = 'cgn_analytics_24_1'
code_df = get_dataframe(f"Select * from {schema}.mindscape_code_dict")
code_df = code_df.to_dict(orient='records')
code_dict = {d['country']: d['code'] for d in code_df}

In [4]:
all_connected = pd.read_csv('data/all_connected.csv')
female_connected = pd.read_csv('data/female_connected.csv')
male_connected = pd.read_csv('data/male_connected.csv')
all_lonely = pd.read_csv('data/all_lonely.csv')
female_lonely = pd.read_csv('data/female_lonely.csv')
male_lonely = pd.read_csv('data/male_lonely.csv')
dfs_all = [all_connected, all_lonely, female_connected, female_lonely, male_connected, male_lonely]  
dfs_names = ['all_connected', 'all_lonely', 'female_connected', 'female_lonely', 'male_connected', 'male_lonely']  

In [5]:
# Defining functions for data cleaning

def delete_percent(df):
    #for index, row in df.iterrows():
    for col in df.columns:
        if col != 'country':
            df[col] = df[col].str.strip('%').astype(float)
    return df

def add_prefix_to_columns(df, prefix):
    new_columns = []
    for col in df.columns:
        if col != 'country' and col != df.index.name:
            if not col.startswith(prefix):
                new_columns.append(prefix + col)
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

def change_columns(df):
    df.columns=df.columns.str.lower().str.replace(" ", "_")

def assign_code(df):
    df['code'] = df['country'].map(code_dict)

In [6]:
for i in dfs_all:
    change_columns(i)
    delete_percent(i)

In [7]:
for i, ii in zip(dfs_all, dfs_names):
        if 'female' in ii:
            add_prefix_to_columns(i, 'female_')
        elif 'male' in ii and 'female' not in ii :
            add_prefix_to_columns(i, 'male_')

In [8]:
merged_lon = pd.merge(all_lonely, female_lonely, on='country', how='outer')
merged_lonely = pd.merge(merged_lon, male_lonely, on='country', how='outer')
merged_con = pd.merge(all_connected, female_connected, on='country', how='outer')
merged_connected = pd.merge(merged_con, male_connected, on='country', how='outer')

In [9]:
assign_code(merged_lonely)
assign_code(merged_connected)

In [10]:
merged_connected = merged_connected[['country', 'code', 'very+fairly', 'a_little_connected', 'not_at_all_connected',
       'female_very+fairly', 'female_a_little_connected',
       'female_not_at_all_connected', 'male_very+fairly',
       'male_a_little_connected', 'male_not_at_all_connected']]
merged_lonely = merged_lonely[['country', 'code', 'very+fairly', 'a_little_lonely', 'not_at_all_lonely',
       'female_very+fairly', 'female_a_little_lonely',
       'female_not_at_all_lonely', 'male_very+fairly', 'male_a_little_lonely',
       'male_not_at_all_lonely']]

In [41]:
schema = 'cgn_analytics_24_1' 
engine = get_engine() 

def export_table(df, name):
    global engine
    table_name = f'mindscape_{name}'
    if engine!=None:
        try:
            df.to_sql(name=table_name, # Name of SQL table
                    con=engine, # Engine or connection
                    if_exists='replace', # Drop the table before inserting new values 
                schema=schema, # Use schmea that was defined earlier
                    index=False, # Write DataFrame index as a column
                    chunksize=5000, # Specify the number of rows in each batch to be written at a time
                    method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was imported successfully.")
    # Error handling
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            engine = None

In [42]:
export_table(merged_connected, '_connected')
export_table(merged_lonely, '_lonely')

The mindscape__connected table was imported successfully.
The mindscape__lonely table was imported successfully.


In [14]:
merged_connected.sort_values(by='very+fairly', ascending=False).head(10)

,country,code,very+fairly,a_little_connected,not_at_all_connected,female_very+fairly,female_a_little_connected,female_not_at_all_connected,male_very+fairly,male_a_little_connected,male_not_at_all_connected
83,Mongolia,MNG,95.0,4.0,1.0,94.0,5.0,1.0,96.0,4.0,1.0
63,Kosovo,XKX,93.0,6.0,1.0,94.0,6.0,0.0,93.0,6.0,1.0
122,"Taiwan, Province of China",NaN,92.0,6.0,2.0,92.0,5.0,2.0,92.0,7.0,1.0
115,Slovenia,SVN,91.0,8.0,1.0,92.0,7.0,1.0,90.0,9.0,0.0
120,Sweden,SWE,90.0,8.0,2.0,93.0,7.0,0.0,88.0,8.0,3.0
59,Japan,JPN,89.0,8.0,2.0,90.0,7.0,2.0,88.0,9.0,2.0
55,Israel,ISR,89.0,9.0,2.0,90.0,8.0,2.0,87.0,10.0,3.0
6,Austria,AUT,89.0,8.0,4.0,89.0,7.0,3.0,87.0,8.0,5.0
49,Iceland,ISL,89.0,10.0,1.0,90.0,8.0,2.0,89.0,11.0,0.0
90,Netherlands,NLD,88.0,9.0,2.0,92.0,7.0,1.0,85.0,12.0,3.0


In [10]:
merged_connected.describe()

,very+fairly,a_little_connected,not_at_all_connected,female_very+fairly,female_a_little_connected,female_not_at_all_connected,male_very+fairly,male_a_little_connected,male_not_at_all_connected
count,142.000000,142.000000,142.00000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000
mean,73.028169,20.070423,6.06338,72.809859,20.056338,6.204225,73.288732,19.929577,5.866197
std,11.384912,8.328919,3.70205,12.451286,9.050508,4.076465,11.012296,8.097498,3.671297
min,47.000000,4.000000,1.00000,42.000000,5.000000,0.000000,47.000000,4.000000,0.000000
25%,65.000000,13.000000,3.00000,64.250000,13.000000,3.000000,66.000000,13.000000,3.000000
50%,75.000000,19.500000,5.00000,75.000000,18.500000,5.000000,74.000000,20.000000,5.000000
75%,82.750000,25.000000,8.00000,83.000000,25.750000,9.000000,83.000000,24.000000,8.000000
max,95.000000,43.000000,19.00000,94.000000,45.000000,21.000000,96.000000,42.000000,19.000000


In [11]:
merged_lonely.describe()

,very+fairly,a_little_lonely,not_at_all_lonely,female_very+fairly,female_a_little_lonely,female_not_at_all_lonely,male_very+fairly,male_a_little_lonely,male_not_at_all_lonely
count,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000
mean,22.992958,26.908451,49.387324,23.176056,27.302817,48.654930,22.711268,26.295775,50.246479
std,12.312420,6.624589,13.893132,12.438387,7.073063,13.758322,12.552630,6.870420,14.535936
min,5.000000,9.000000,13.000000,4.000000,9.000000,10.000000,4.000000,9.000000,16.000000
25%,13.000000,22.250000,39.250000,13.250000,22.000000,38.250000,13.000000,21.000000,40.000000
50%,21.000000,26.000000,50.000000,21.000000,26.000000,48.000000,20.000000,26.000000,51.000000
75%,29.750000,30.000000,60.000000,29.000000,31.000000,59.000000,30.000000,30.000000,61.000000
max,58.000000,45.000000,86.000000,56.000000,48.000000,86.000000,62.000000,44.000000,86.000000
